# Bouncing Ball with Manim

Run the cells below to render a bouncing ball animation using Manim inside Jupyter.


In [34]:
%load_ext manim.utils.ipython_magic

The manim.utils.ipython_magic module is not an IPython extension.


In [35]:
import json
import os
from pathlib import Path
simulation_params = {
    "floor_y": -2.7,  # scene floor position in Manim coordinates
    "gravity": 9.8,  # acceleration due to gravity (units/s^2)
    "initial_height": 10.0,  # starting drop height in physics units
    "initial_velocity": 0.0,  # initial vertical speed of the ball
    "total_time": 20.0,  # total animation runtime in seconds
    "simulation_speed": 4.0,  # how many simulated seconds advance per animation second
    "graph_point_limit": 300,  # number of recent samples to show on the graph
    "energy_threshold": 5.0,  # stop once total energy drops below this value
    "coefficient": 0.95,  # coefficient of restitution used for each bounce
}
cwd = Path.cwd().resolve()
if cwd.name == "notebooks" or not (cwd / "notebooks").exists():
    param_dir = cwd
else:
    param_dir = cwd / "notebooks"
param_dir.mkdir(parents=True, exist_ok=True)
param_file = param_dir / "simulation_params.json"
param_file.write_text(json.dumps(simulation_params))
os.environ["BOUNCING_PARAMS_PATH"] = str(param_file)


In [36]:
%%manim -qm -v WARNING BouncingBallWithGraph
from manim import *
import numpy as np
import math
import json
import os
from pathlib import Path
def load_simulation_params():
    """Load parameters written by the companion cell or fall back to defaults."""
    default_params = {
        "floor_y": -2.7,
        "gravity": 9.8,
        "initial_height": 10.0,
        "initial_velocity": 0.0,
        "total_time": 20.0,
        "simulation_speed": 4.0,
        "graph_point_limit": 300,
        "energy_threshold": 5.0,
        "coefficient": 0.95,
    }
    env_path = os.environ.get("BOUNCING_PARAMS_PATH")
    if env_path:
        try:
            loaded = json.loads(Path(env_path).read_text())
            return {**default_params, **loaded}
        except OSError:
            pass
    return default_params
class BouncingBallWithGraph(Scene):
    def construct(self):
        params = load_simulation_params()
        floor_y = params["floor_y"]
        gravity = params["gravity"]
        initial_height = params["initial_height"]
        initial_velocity = params["initial_velocity"]
        total_time = params["total_time"]
        simulation_speed = params["simulation_speed"]
        graph_point_limit = max(10, int(params["graph_point_limit"]))
        energy_threshold = params["energy_threshold"]
        # ValueTracker lets the viewer adjust restitution during the run.
        coefficient = ValueTracker(params["coefficient"])
        # Static ground reference and restitution readout.
        ground = Line(LEFT * 2.5, RIGHT * 0.5).shift(DOWN * 2.7 + LEFT * 3.5)
        restitution_label = VGroup(
            Text("Restitution").scale(0.45),
            DecimalNumber(num_decimal_places=2),
        ).arrange(RIGHT, buff=0.2).next_to(ground, UP, buff=0.2).shift(LEFT * 1.2)
        restitution_label[1].add_updater(lambda d: d.set_value(coefficient.get_value()))
        # Trackers retain the current simulation state for the updaters and graph.
        time_tracker = ValueTracker(0.0)
        height_tracker = ValueTracker(initial_height)
        velocity_tracker = ValueTracker(initial_velocity)
        initial_energy = gravity * initial_height + 0.5 * initial_velocity**2
        energy_tracker = ValueTracker(initial_energy)
        # Create the ball and visual shadow, mapping physics height to scene space.
        ball = Circle(radius=0.3, color=BLUE).set_fill(BLUE, opacity=0.9)
        ball_radius = ball.radius
        scene_top = 3.2
        usable_scene_height = scene_top - (floor_y + ball_radius)
        height_scale = usable_scene_height / max(initial_height, 1.0)
        ball.move_to(np.array([-3.5, floor_y + ball_radius + initial_height * height_scale, 0]))
        shadow = Circle(radius=0.22, color=GRAY_E, fill_opacity=0.4)
        def update_shadow(mob):
            # Shadow squashes as the ball approaches the ground.
            height = height_tracker.get_value()
            scale = np.interp(height, [0, initial_height], [1.4, 0.75])
            mob.become(
                Circle(radius=0.22, color=GRAY_E, fill_opacity=0.4)
                .move_to(np.array([-3.5, floor_y + 0.02, 0]))
                .scale(scale)
            )
        shadow.add_updater(update_shadow)
        height_history = [(0.0, initial_height)]
        def get_recent_points():
            window = height_history[-graph_point_limit:] or [(0.0, initial_height)]
            offset = window[0][0]
            shifted = [(t - offset, h) for t, h in window]
            return shifted, offset

        def update_ball(mob, dt):
            """Semi-implicit integrator with exact bounce time detection."""
            scaled_dt = dt * simulation_speed
            height = height_tracker.get_value()
            velocity = velocity_tracker.get_value()
            remaining_dt = scaled_dt
            while remaining_dt > 1e-6:
                time_to_floor = None
                if velocity < 0:
                    a = -0.5 * gravity
                    b = velocity
                    c = height
                    discriminant = b * b - 4 * a * c
                    if discriminant >= 0:
                        sqrt_disc = math.sqrt(discriminant)
                        potential_hit = (-b - sqrt_disc) / (2 * a)
                        if 0 <= potential_hit <= remaining_dt:
                            time_to_floor = potential_hit
                if time_to_floor is not None:
                    # Integrate down to the exact collision time, flip velocity, and lose energy.
                    height = height + velocity * time_to_floor - 0.5 * gravity * time_to_floor**2
                    velocity = velocity - gravity * time_to_floor
                    height = 0.0
                    velocity = -velocity * coefficient.get_value()
                    remaining_dt -= time_to_floor
                else:
                    # No collision in this sub-step: integrate normally.
                    height = height + velocity * remaining_dt - 0.5 * gravity * remaining_dt**2
                    velocity = velocity - gravity * remaining_dt
                    remaining_dt = 0.0
            height = max(height, 0.0)
            time_tracker.increment_value(scaled_dt)
            current_time = time_tracker.get_value()
            height_tracker.set_value(height)
            velocity_tracker.set_value(velocity)
            potential_energy = gravity * height
            kinetic_energy = 0.5 * velocity**2
            total_energy = potential_energy + kinetic_energy
            energy_tracker.set_value(total_energy)
            height_history.append((current_time, height))
            if len(height_history) > graph_point_limit:
                del height_history[0: len(height_history) - graph_point_limit]
            if total_energy < energy_threshold:
                # Stop all updaters once we are effectively at rest.
                height_tracker.set_value(0.0)
                velocity_tracker.set_value(0.0)
                energy_tracker.set_value(0.0)
                height_history.append((current_time, 0.0))
                mob.move_to(np.array([mob.get_x(), floor_y + ball_radius, 0]))
                mob.remove_updater(update_ball)
                return
            mob.move_to(
                np.array([
                    mob.get_x(),
                    floor_y + ball_radius + height * height_scale,
                    0,
                ])
            )
        ball.add_updater(update_ball)
        def build_axes():
            # Keep the axes window focused on the most recent graph_point_limit samples.
            recent_points, _ = get_recent_points()
            span = recent_points[-1][0] if len(recent_points) > 1 else 1.0
            if span < 1e-3:
                span = 1.0
            y_values = [h for _, h in recent_points]
            y_extent = max(1.0, max(y_values) * 1.1)
            x_step = max(1.0, span / 4)
            y_step = max(0.5, y_extent / 5)
            axes = Axes(
                x_range=[0, span, x_step],
                y_range=[0, y_extent, y_step],
                x_length=5,
                y_length=4,
                axis_config={"include_ticks": False},
            ).to_corner(UR).shift(LEFT * 0.5)
            labels = axes.get_axis_labels(Tex("t (s)"), Tex("height"))
            return axes, labels

        axes, axes_labels = build_axes()
        axes_group = VGroup(axes, axes_labels)

        def update_axes_group(group):
            new_axes, new_labels = build_axes()
            group.become(VGroup(new_axes, new_labels))

        axes_group.add_updater(update_axes_group)

        def history_points_for_graph():
            # Only keep the last graph_point_limit samples for plotting.
            return get_recent_points()[0]

        velocity_arrow = always_redraw(
            lambda: Arrow(
                start=ball.get_center(),
                end=ball.get_center() + UP * velocity_tracker.get_value() * height_scale * 0.12,
                buff=0,
                color=YELLOW,
                max_tip_length_to_length_ratio=0.15,
                stroke_width=5,
            )
        )
        graph = always_redraw(
            lambda: (
                lambda pts: axes_group[0].plot_line_graph(
                    [t for t, _ in pts] if pts else [0, 0.0001],
                    [h for _, h in pts] if pts else [initial_height, initial_height],
                    line_color=BLUE,
                    add_vertex_dots=False,
                )
            )(history_points_for_graph())
        )
        graph_dot = always_redraw(
            lambda: (
                lambda offset: Dot(
                    axes_group[0].c2p(max(0.0, time_tracker.get_value() - offset), height_tracker.get_value()),
                    color=YELLOW,
                    radius=0.06,
                )
            )(get_recent_points()[1])
        )

        max_energy = max(initial_energy, 1e-3)
        energy_scale = 2.5 / max_energy
        energy_base_point = LEFT * 5.3 + DOWN * 0.4
        def build_energy_bar():
            # Total energy = bar height; kinetic energy fills the red segment from the bottom.
            total_energy = energy_tracker.get_value()
            kinetic_energy = 0.5 * velocity_tracker.get_value() ** 2
            total_height = max(0.05, total_energy * energy_scale)
            kinetic_height = min(total_height, kinetic_energy * energy_scale)
            bar_width = 0.5
            total_rect = Rectangle(
                width=bar_width,
                height=total_height,
                stroke_color=WHITE,
                stroke_width=2,
                fill_color=WHITE,
                fill_opacity=1,
            ).move_to(energy_base_point + UP * (total_height / 2))
            kinetic_rect = Rectangle(
                width=bar_width,
                height=kinetic_height,
                stroke_width=0,
                fill_color=RED,
                fill_opacity=0.9,
            ).move_to(energy_base_point + UP * (kinetic_height / 2))
            return VGroup(total_rect, kinetic_rect)
        energy_bar = always_redraw(build_energy_bar)
        energy_label = Text("Energy").scale(0.35).next_to(energy_bar, DOWN, buff=0.2)
        self.play(
            Create(ground),
            FadeIn(ball),
            FadeIn(shadow),
            Create(axes_group[0]),
            FadeIn(axes_group[1]),
            FadeIn(restitution_label),
            FadeIn(energy_bar),
            FadeIn(energy_label),
        )
        self.add(axes_group, graph, graph_dot, velocity_arrow, energy_bar, energy_label)
        self.wait(
            total_time,
            stop_condition=lambda: energy_tracker.get_value() < energy_threshold,
        )
        self.play(
            FadeOut(ball),
            FadeOut(shadow),
            FadeOut(ground),
            FadeOut(axes_group),
            FadeOut(graph),
            FadeOut(graph_dot),
            FadeOut(restitution_label),
            FadeOut(velocity_arrow),
            FadeOut(energy_bar),
            FadeOut(energy_label),
        )
        self.wait(0.2)


Manim Community v0.19.0